In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!nvidia-smi

Fri Nov  8 08:42:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 43.6 MB/s eta 0:00:00


In [5]:
%cd /content/drive/MyDrive/yolo training

/content/drive/MyDrive/yolo training


In [25]:
#!git clone https://github.com/autogyro/yolo-V8.git


Cloning into 'yolo-V8'...
remote: Enumerating objects: 2723, done.
remote: Total 2723 (delta 0), reused 0 (delta 0), pack-reused 2723 (from 1)
Receiving objects: 100% (2723/2723), 1.41 MiB | 2.83 MiB/s, done.
Resolving deltas: 100% (1855/1855), done.
Updating files: 100% (161/161), done.


In [6]:
import os
os.chdir('/content/drive/MyDrive/yolo training/yolo-V8')

In [7]:
ls

CITATION.cff     docker/    images_to_train/  mkdocs.yml       requirements.txt  tests/
CONTRIBUTING.md  docs/      LICENSE           README.md        setup.cfg         training_results/
data.yaml        examples/  MANIFEST.in       README.zh-CN.md  setup.py          ultralytics/


In [32]:
#!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.2 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=ed3299013b7b4bc2ea653a429d0ab5e9b337e56ebb154f3d1ca079a01cd2cb3e
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


# checking and removing the images having no annotations

In [13]:
import os

# Folders to check
train_folder = '/content/drive/MyDrive/yolo training/yolo-V8/images_to_train/train'
val_folder = '/content/drive/MyDrive/yolo training/yolo-V8/images_to_train/val'

# List to store images to be deleted
images_to_delete = []

# Function to check and collect images without annotations
def check_and_delete_images(folder):
    for img_file in os.listdir(folder):
        if img_file.endswith(('.jpg', '.png')):  # Check image files
            base_name = os.path.splitext(img_file)[0]
            txt_file = os.path.join(folder, base_name + '.txt')
            if not os.path.exists(txt_file):
                images_to_delete.append(img_file)

# Check both train and val directories
check_and_delete_images(train_folder)
check_and_delete_images(val_folder)

# Remove images that have no annotations
if images_to_delete:
    print("Removing the following images (no annotations found):")
    for img in images_to_delete:
        img_path = os.path.join(train_folder, img) if img in os.listdir(train_folder) else os.path.join(val_folder, img)
        os.remove(img_path)  # Remove image file
        print(f"Removed: {img}")
else:
    print("No images to remove.")


Removing the following images (no annotations found):
Removed: 2011_002805_jpg.rf.0b75ae8acc7ed12bb2751f86b8e2c768 (1).jpg
Removed: 2011_004254_jpg.rf.06fbe43fa0e407bcc33ff004ebe5cdc1 (1).jpg
Removed: 2011_004086_jpg.rf.895a71205e95ebf29c8360244f8e37ec (1).jpg


# training the image of train folder

In [1]:
!yolo task=detect mode=train model=yolov8n.pt data="/content/drive/MyDrive/yolo training/yolo-V8/data.yaml" epochs=10 imgsz=640 batch=16 plots=True project="/content/drive/MyDrive/yolo training/yolo-V8/training_results" name="train_run_2"


/bin/bash: line 1: yolo: command not found


# training the images of validation folder

In [9]:
!yolo task=detect mode=val model="/content/drive/MyDrive/yolo training/yolo-V8/training_results/train_run_2/weights/best.pt" data="/content/drive/MyDrive/yolo training/yolo-V8/data.yaml" imgsz=640 project="/content/drive/MyDrive/yolo training/yolo-V8/validation_results" name="val_run_1"


Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/yolo training/yolo-V8/images_to_train/val.cache... 3328 images, 0 backgrounds, 0 corrupt: 100% 3328/3328 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 208/208 [01:59<00:00,  1.74it/s]
                   all       3328       7687      0.562      0.553      0.546      0.379
                person       1784       3243      0.674      0.739      0.723      0.507
                 chair        262        566      0.557      0.351      0.402      0.276
                   car        258        509      0.611      0.537      0.575      0.417
                   dog        297        353       0.83       0.34      0.619      0.443
                bottle        154        329      0.628      0.261       0.36      0.235
                

In [6]:
!yolo task=detect mode=predict model="/content/drive/MyDrive/yolo training/yolo-V8/training_results/train_run_2/weights/best.pt" source="/content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/sampleImages" imgsz=640 --project="/content/drive/MyDrive/yolo training/yolo-V8/test_results"


WARNING ⚠️ argument '--project=/content/drive/MyDrive/yolo training/yolo-V8/test_results' does not require leading dashes '--', updating to 'project=/content/drive/MyDrive/yolo training/yolo-V8/test_results'.
Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs

image 1/312 /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/sampleImages/0001.jpg: 448x640 1 car, 153.7ms
image 2/312 /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/sampleImages/00010.jpeg: 384x640 1 car, 133.4ms
image 3/312 /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/sampleImages/00010.jpg: 448x640 3 persons, 1 car, 182.9ms
image 4/312 /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/sampleImages/00011.jpeg: 480x640 1 person, 1 car, 273.3ms
image 5/312 /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/sampleImages/00012.jpg: 448x640 1 car, 282.4ms
image 6/

# check the model on the random video

In [9]:
!yolo task=detect mode=predict model="/content/drive/MyDrive/yolo training/yolo-V8/training_results/train_run_2/weights/best.pt" source="/content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/WhatsApp Video 2024-11-09 at 20.22.44_9ca26a84.mp4" imgsz=640 project="/content/drive/MyDrive/yolo training/yolo-V8/test_results" name="test_run_1"


Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (frame 1/684) /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/WhatsApp Video 2024-11-09 at 20.22.44_9ca26a84.mp4: 640x384 15 cars, 150.1ms
video 1/1 (frame 2/684) /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/WhatsApp Video 2024-11-09 at 20.22.44_9ca26a84.mp4: 640x384 16 cars, 135.4ms
video 1/1 (frame 3/684) /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/WhatsApp Video 2024-11-09 at 20.22.44_9ca26a84.mp4: 640x384 1 person, 16 cars, 124.1ms
video 1/1 (frame 4/684) /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/WhatsApp Video 2024-11-09 at 20.22.44_9ca26a84.mp4: 640x384 14 cars, 125.2ms
video 1/1 (frame 5/684) /content/drive/MyDrive/yolo training/yolo-V8/IMAGES_FOR TEST/WhatsApp Video 2024-11-09 at 20.22.44_9ca26a84.mp4: 640x384 17 cars, 118.8ms
video 1/1 (frame 6/